In [ ]:
from pymilvus import CollectionSchema, FieldSchema, DataType, Collection, connections, MilvusException
import json
from ctransformers import AutoModelForCausalLM
import spacy

In [ ]:
##Collection config
id = FieldSchema(
  name="id",
  dtype=DataType.INT64,
  is_primary=True,
  auto_id=True
)
text = FieldSchema(
  name="text",
  dtype=DataType.VARCHAR,
  max_length=5000,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)
vector = FieldSchema(
  name="vector",
  dtype=DataType.FLOAT_VECTOR,
  dim=128
)
schema = CollectionSchema(
  fields=[id, text, vector],
  enable_dynamic_field=True
)
collection_name = "Products"

In [ ]:
##connect to milvus server
connections.connect(host="localhost", port="19530")

In [ ]:
##Create collection
collection = Collection(
    name=collection_name,
    schema=schema,
    using='default', # Milvus server alias
    shards_num=2 # Number of data nodes to use
    )

In [ ]:
##Add index to vector
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":2}
}

from pymilvus import Collection, utility
# Get an existing collection.
collection = Collection("Products")      
collection.create_index(
  field_name="vector", 
  index_params=index_params
)

utility.index_building_progress("Products")

In [ ]:
# Get an existing collection and load
collection = Collection("Products")      
collection.load()

# Check the loading progress and loading status
utility.load_state("Products")
# Output: <LoadState: Loaded>

utility.loading_progress("Products")
# Output: {'loading_progress': 100%}

In [ ]:
##prepare data for dabase and upload
with open('Product_data.json','r') as f:
    products = json.load(f)  

data = []
text = []
vector = []
for ele in products:
    text.append(ele['text'])
    vector.append(ele['vector'])

data.append(text)
data.append(vector)

from pymilvus import Collection
collection = Collection("Products")      # Get an existing collection.
mr = collection.insert(data)

In [ ]:
## Lists existing collections
utility.list_collections()

## Inference

In [ ]:
##Load LLM model
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("llama-2-7b-chat.Q2_K.gguf", model_file="llama-2-7b-chat.Q2_K.gguf",model_type="llama", gpu_layers=0,context_length=4096)

# Load the spaCy model for English language
spacy_model = spacy.load('en_core_web_lg')

# Connect to Milvus server
connections.connect(host="localhost", port="19530")


query = 'What was the conclusion on the discussion about the road contract'

In [ ]:
## Run inference in loop for different queries
try:
    while True:
        user_input = input("\nDescribe query (or type 'exit' to quit):\n")
        print(f"User input : {user_input}\n")

        # Exit loop if user types 'exit'
        if user_input.lower() == 'exit':
            break

        # Process user input using spaCy model to get embedding vector
        user_input_doc = spacy_model(user_input)
        user_vector = user_input_doc.vector[:128].tolist()

        # Define search parameters for similarity search
        search_params = {
            "metric_type": "L2",
            "offset": 0, # Number of entities to skip during the search
            "ignore_growing": False, # Whether to ignore growing segments during similarity searches
            "params": {"nprobe": 1}
        }

        # Connect to the Milvus collection named "Products"
        collection = Collection("Products")

        # Perform similarity search using Milvus
        similarity_search_result = collection.search(
            data=[user_vector],
            anns_field="vector",
            param=search_params,
            limit=1,
            output_fields=['text']
        )

        # Display search results to the user
        # for idx, hit in enumerate(similarity_search_result[0]):
        score = similarity_search_result[0][0].distance
        description = similarity_search_result[0][0].entity.text
        print(f"{description} \ndistance: {score}\n")

        prompt = f"Use context to answer the query.\n context : {description}\n query:{user_input}\n answer : "
        input_text = f"[INST] <<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>[/INST] \n{prompt}"
        print(f'LLM response : {llm(prompt)}\n')

except MilvusException as e:
    # Handle Milvus exceptions
    print(e)
finally:
    # Disconnect from Milvus server
    connections.disconnect(alias="localhost")